<a href="https://colab.research.google.com/github/vsay01/deep-learning-v2-pytorch/blob/develop/intro-to-pytorch/Part%201%20-%20Tensors%20in%20PyTorch%20(Exercises).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Deep Learning with PyTorch

In this notebook, you'll get introduced to [PyTorch](http://pytorch.org/), a framework for building and training neural networks. PyTorch in a lot of ways behaves like the arrays you love from Numpy. These Numpy arrays, after all, are just tensors. PyTorch takes these tensors and makes it simple to move them to GPUs for the faster processing needed when training neural networks. It also provides a module that automatically calculates gradients (for backpropagation!) and another module specifically for building neural networks. All together, PyTorch ends up being more coherent with Python and the Numpy/Scipy stack compared to TensorFlow and other frameworks.



## Neural Networks

Deep Learning is based on artificial neural networks which have been around in some form since the late 1950s. The networks are built from individual parts approximating neurons, typically called units or simply "neurons." Each unit has some number of weighted inputs. These weighted inputs are summed together (a linear combination) then passed through an activation function to get the unit's output.

<img src="https://github.com/vsay01/deep-learning-v2-pytorch/blob/develop/intro-to-pytorch/assets/simple_neuron.png?raw=1" width=400px>

Mathematically this looks like: 

$$
\begin{align}
y &= f(w_1 x_1 + w_2 x_2 + b) \\
y &= f\left(\sum_i w_i x_i +b \right)
\end{align}
$$

With vectors this is the dot/inner product of two vectors:

$$
h = \begin{bmatrix}
x_1 \, x_2 \cdots  x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_1 \\
           w_2 \\
           \vdots \\
           w_n
\end{bmatrix}
$$

## Tensors

It turns out neural network computations are just a bunch of linear algebra operations on *tensors*, a generalization of matrices. A vector is a 1-dimensional tensor, a matrix is a 2-dimensional tensor, an array with three indices is a 3-dimensional tensor (RGB color images for example). The fundamental data structure for neural networks are tensors and PyTorch (as well as pretty much every other deep learning framework) is built around tensors.

<img src="https://github.com/vsay01/deep-learning-v2-pytorch/blob/develop/intro-to-pytorch/assets/tensor_examples.svg?raw=1" width=600px>

With the basics covered, it's time to explore how we can use PyTorch to build a simple neural network.

In [0]:
# First, import PyTorch
import torch

In [0]:
def activation(x):
    """ Sigmoid activation function 
    
        Arguments
        ---------
        x: torch.Tensor
    """
    return 1/(1+torch.exp(-x))

In [0]:
### Generate some data
torch.manual_seed(7) # Set the random seed so things are predictable

# Features are 5 random normal variables
features = torch.randn((1, 5))
# True weights for our data, random normal variables again
weights = torch.randn_like(features)
# and a true bias term
bias = torch.randn((1, 1))

Above I generated data we can use to get the output of our simple network. This is all just random for now, going forward we'll start using normal data. Going through each relevant line:

`features = torch.randn((1, 5))` creates a tensor with shape `(1, 5)`, one row and five columns, that contains values randomly distributed according to the normal distribution with a mean of zero and standard deviation of one. 

`weights = torch.randn_like(features)` creates another tensor with the same shape as `features`, again containing values from a normal distribution.

Finally, `bias = torch.randn((1, 1))` creates a single value from a normal distribution.

PyTorch tensors can be added, multiplied, subtracted, etc, just like Numpy arrays. In general, you'll use PyTorch tensors pretty much the same way you'd use Numpy arrays. They come with some nice benefits though such as GPU acceleration which we'll get to later. For now, use the generated data to calculate the output of this simple single layer network. 
> **Exercise**: Calculate the output of the network with input features `features`, weights `weights`, and bias `bias`. Similar to Numpy, PyTorch has a [`torch.sum()`](https://pytorch.org/docs/stable/torch.html#torch.sum) function, as well as a `.sum()` method on tensors, for taking sums. Use the function `activation` defined above as the activation function.

In [28]:
## Calculate the output of this network using the weights and bias tensors
print(features)
print(features.size())
print(weights)
print(weights.size())
y = activation(torch.sum(features * weights) + bias)
y = activation((features * weights).sum() + bias)

torch.mm(features, weights)

tensor([[-0.1468,  0.7861,  0.9468, -1.1143,  1.6908]])
torch.Size([1, 5])
tensor([[-0.8948, -0.3556,  1.2324,  0.1382, -1.6822]])
torch.Size([1, 5])


RuntimeError: ignored

You can do the multiplication and sum in the same operation using a matrix multiplication. In general, you'll want to use matrix multiplications since they are more efficient and accelerated using modern libraries and high-performance computing on GPUs.

Here, we want to do a matrix multiplication of the features and the weights. For this we can use [`torch.mm()`](https://pytorch.org/docs/stable/torch.html#torch.mm) or [`torch.matmul()`](https://pytorch.org/docs/stable/torch.html#torch.matmul) which is somewhat more complicated and supports broadcasting. If we try to do it with `features` and `weights` as they are, we'll get an error

```python
>> torch.mm(features, weights)

---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-13-15d592eb5279> in <module>()
----> 1 torch.mm(features, weights)

RuntimeError: size mismatch, m1: [1 x 5], m2: [1 x 5] at /Users/soumith/minicondabuild3/conda-bld/pytorch_1524590658547/work/aten/src/TH/generic/THTensorMath.c:2033
```

As you're building neural networks in any framework, you'll see this often. Really often. What's happening here is our tensors aren't the correct shapes to perform a matrix multiplication. Remember that for matrix multiplications, the number of columns in the first tensor must equal to the number of rows in the second column. Both `features` and `weights` have the same shape, `(1, 5)`. This means we need to change the shape of `weights` to get the matrix multiplication to work.

**Note:** To see the shape of a tensor called `tensor`, use `tensor.shape`. If you're building neural networks, you'll be using this method often.

There are a few options here: [`weights.reshape()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.reshape), [`weights.resize_()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.resize_), and [`weights.view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view).

* `weights.reshape(a, b)` will return a new tensor with the same data as `weights` with size `(a, b)` sometimes, and sometimes a clone, as in it copies the data to another part of memory.
* `weights.resize_(a, b)` returns the same tensor with a different shape. However, if the new shape results in fewer elements than the original tensor, some elements will be removed from the tensor (but not from memory). If the new shape results in more elements than the original tensor, new elements will be uninitialized in memory. Here I should note that the underscore at the end of the method denotes that this method is performed **in-place**. Here is a great forum thread to [read more about in-place operations](https://discuss.pytorch.org/t/what-is-in-place-operation/16244) in PyTorch.
* `weights.view(a, b)` will return a new tensor with the same data as `weights` with size `(a, b)`.

I usually use `.view()`, but any of the three methods will work for this. So, now we can reshape `weights` to have five rows and one column with something like `weights.view(5, 1)`.

> **Exercise**: Calculate the output of our little network using matrix multiplication.

In [29]:
## Calculate the output of this network using matrix multiplication
activation(torch.mm(features, weights.view(5, 1)) + bias)

tensor([[0.1595]])

### Stack them up!

That's how you can calculate the output for a single neuron. The real power of this algorithm happens when you start stacking these individual units into layers and stacks of layers, into a network of neurons. The output of one layer of neurons becomes the input for the next layer. With multiple input units and output units, we now need to express the weights as a matrix.

<img src='https://github.com/vsay01/deep-learning-v2-pytorch/blob/develop/intro-to-pytorch/assets/multilayer_diagram_weights.png?raw=1' width=450px>

The first layer shown on the bottom here are the inputs, understandably called the **input layer**. The middle layer is called the **hidden layer**, and the final layer (on the right) is the **output layer**. We can express this network mathematically with matrices again and use matrix multiplication to get linear combinations for each unit in one operation. For example, the hidden layer ($h_1$ and $h_2$ here) can be calculated 

$$
\vec{h} = [h_1 \, h_2] = 
\begin{bmatrix}
x_1 \, x_2 \cdots \, x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_{11} & w_{12} \\
           w_{21} &w_{22} \\
           \vdots &\vdots \\
           w_{n1} &w_{n2}
\end{bmatrix}
$$

The output for this small network is found by treating the hidden layer as inputs for the output unit. The network output is expressed simply

$$
y =  f_2 \! \left(\, f_1 \! \left(\vec{x} \, \mathbf{W_1}\right) \mathbf{W_2} \right)
$$

In [0]:
### Generate some data
torch.manual_seed(7) # Set the random seed so things are predictable

# Features are 3 random normal variables
features = torch.randn((1, 3))

# Define the size of each layer in our network
n_input = features.shape[1]     # Number of input units, must match number of input features
n_hidden = 2                    # Number of hidden units 
n_output = 1                    # Number of output units

# Weights for inputs to hidden layer
W1 = torch.randn(n_input, n_hidden)
# Weights for hidden layer to output layer
W2 = torch.randn(n_hidden, n_output)

# and bias terms for hidden and output layers
B1 = torch.randn((1, n_hidden))
B2 = torch.randn((1, n_output))

> **Exercise:** Calculate the output for this multi-layer network using the weights `W1` & `W2`, and the biases, `B1` & `B2`. 

In [31]:
## Your solution here
print(features)
print(features.size())

print(W1)
print(W1.size())

print(W2)
print(W2.size())

print(B1)
print(B1.size())

print(B2)
print(B2.size())

# h = activation(torch.mm(features, W1) + B1)
# output = activation(torch.mm(h, W2) + B2)
activation(torch.mm(activation(torch.mm(features, W1) + B1), W2) + B2)

tensor([[-0.1468,  0.7861,  0.9468]])
torch.Size([1, 3])
tensor([[-1.1143,  1.6908],
        [-0.8948, -0.3556],
        [ 1.2324,  0.1382]])
torch.Size([3, 2])
tensor([[-1.6822],
        [ 0.3177]])
torch.Size([2, 1])
tensor([[0.1328, 0.1373]])
torch.Size([1, 2])
tensor([[0.2405]])
torch.Size([1, 1])


tensor([[0.3171]])

If you did this correctly, you should see the output `tensor([[ 0.3171]])`.

The number of hidden units a parameter of the network, often called a **hyperparameter** to differentiate it from the weights and biases parameters. As you'll see later when we discuss training a neural network, the more hidden units a network has, and the more layers, the better able it is to learn from data and make accurate predictions.

## Numpy to Torch and back

Special bonus section! PyTorch has a great feature for converting between Numpy arrays and Torch tensors. To create a tensor from a Numpy array, use `torch.from_numpy()`. To convert a tensor to a Numpy array, use the `.numpy()` method.

In [32]:
import numpy as np
a = np.random.rand(4,3)
a

array([[0.10484639, 0.80603393, 0.75629853],
       [0.37954851, 0.35750601, 0.4294709 ],
       [0.45801771, 0.17299435, 0.85289598],
       [0.78605566, 0.623105  , 0.44463525]])

In [33]:
b = torch.from_numpy(a)
b

tensor([[0.1048, 0.8060, 0.7563],
        [0.3795, 0.3575, 0.4295],
        [0.4580, 0.1730, 0.8529],
        [0.7861, 0.6231, 0.4446]], dtype=torch.float64)

In [34]:
b.numpy()

array([[0.10484639, 0.80603393, 0.75629853],
       [0.37954851, 0.35750601, 0.4294709 ],
       [0.45801771, 0.17299435, 0.85289598],
       [0.78605566, 0.623105  , 0.44463525]])

The memory is shared between the Numpy array and Torch tensor, so if you change the values in-place of one object, the other will change as well.

In [35]:
# Multiply PyTorch Tensor by 2, in place
b.mul_(2)

tensor([[0.2097, 1.6121, 1.5126],
        [0.7591, 0.7150, 0.8589],
        [0.9160, 0.3460, 1.7058],
        [1.5721, 1.2462, 0.8893]], dtype=torch.float64)

In [36]:
# Numpy array matches new values from Tensor
a

array([[0.20969279, 1.61206786, 1.51259705],
       [0.75909702, 0.71501203, 0.8589418 ],
       [0.91603542, 0.34598869, 1.70579196],
       [1.57211131, 1.24621   , 0.88927049]])

# Practices PyTorch Tensors Operation

In [0]:
# First, import PyTorch
from __future__ import print_function
import torch

In [38]:
# Construct a 5x3 matrix
x = torch.rand(5, 3)
print(x)

tensor([[0.7653, 0.1132, 0.8559],
        [0.6721, 0.6267, 0.5691],
        [0.7437, 0.9592, 0.3887],
        [0.2214, 0.3742, 0.1953],
        [0.7405, 0.2529, 0.2332]])


In [39]:
# Construct a matrix filled zeros and of data type long:
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [40]:
# Construct a tensor directly from data:
x = torch.tensor([5.5, 3])
print(x)

tensor([5.5000, 3.0000])


In [41]:
# Create tensor based on exiting tensor
x = x.new_ones(5, 3, dtype=torch.double)    
print(x)

x = torch.randn_like(x, dtype=torch.float)    
print(x)

# Print size
print(x.size())
torch.Size([5, 3])

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[-1.0587,  0.3344, -0.7804],
        [ 0.0804,  0.5719,  1.5958],
        [ 2.4130,  1.9782,  0.1397],
        [-1.0684,  0.3852, -0.4385],
        [-0.0163,  0.8480, -1.7038]])
torch.Size([5, 3])


torch.Size([5, 3])

In [42]:
# Tensor Addition
# Same dimension

# Syntax 1 for Tensor addition in PyTorch
y = torch.rand(5, 3)
print(x)
print(y)
print(x + y)

# Syntax 2 for Tensor addition in PyTorch
print(torch.add(x, y))

tensor([[-1.0587,  0.3344, -0.7804],
        [ 0.0804,  0.5719,  1.5958],
        [ 2.4130,  1.9782,  0.1397],
        [-1.0684,  0.3852, -0.4385],
        [-0.0163,  0.8480, -1.7038]])
tensor([[0.9717, 0.5921, 0.5352],
        [0.6686, 0.3781, 0.9278],
        [0.0922, 0.3625, 0.4136],
        [0.2943, 0.3762, 0.8099],
        [0.7743, 0.5278, 0.9356]])
tensor([[-0.0870,  0.9265, -0.2451],
        [ 0.7490,  0.9500,  2.5235],
        [ 2.5052,  2.3407,  0.5533],
        [-0.7741,  0.7614,  0.3714],
        [ 0.7580,  1.3758, -0.7682]])
tensor([[-0.0870,  0.9265, -0.2451],
        [ 0.7490,  0.9500,  2.5235],
        [ 2.5052,  2.3407,  0.5533],
        [-0.7741,  0.7614,  0.3714],
        [ 0.7580,  1.3758, -0.7682]])


In [43]:
# Tensor substraction
# Same dimension

# Syntax 1 for Tensor addition in PyTorch
y = torch.rand(5, 3)
print(x)
print(y)
print(x - y)

# Syntax 2 for Tensor addition in PyTorch
print(torch.sub(x, y))

tensor([[-1.0587,  0.3344, -0.7804],
        [ 0.0804,  0.5719,  1.5958],
        [ 2.4130,  1.9782,  0.1397],
        [-1.0684,  0.3852, -0.4385],
        [-0.0163,  0.8480, -1.7038]])
tensor([[0.1470, 0.2019, 0.7259],
        [0.4731, 0.0444, 0.1539],
        [0.1956, 0.8861, 0.7264],
        [0.8868, 0.1278, 0.0125],
        [0.1721, 0.5535, 0.4012]])
tensor([[-1.2056,  0.1325, -1.5063],
        [-0.3927,  0.5275,  1.4418],
        [ 2.2174,  1.0921, -0.5867],
        [-1.9552,  0.2574, -0.4510],
        [-0.1884,  0.2945, -2.1051]])
tensor([[-1.2056,  0.1325, -1.5063],
        [-0.3927,  0.5275,  1.4418],
        [ 2.2174,  1.0921, -0.5867],
        [-1.9552,  0.2574, -0.4510],
        [-0.1884,  0.2945, -2.1051]])


In [44]:
# Experiment add tensor different dimen (different row, column)
x = torch.rand(5,3)
y = torch.rand(3,5)
print(x + y)

RuntimeError: ignored

In [45]:
# Experiment add tensor different dimen (same row, column)
y = torch.rand(5,2)
print(x + y)

RuntimeError: ignored

In [46]:
# Tensor product matrix multiplication
# If mat1 is a (n×m) tensor, mat2 is a (m×p) tensor, out will be a (n×p) tensor.
mat1 = torch.randn(2, 3)
mat2 = torch.randn(3, 3)
print(mat1)
print(mat2)
print(torch.mm(mat1, mat2))
print(torch.mm(mat1, mat2).size())

# torch.mm() does not broadcast. What is that?

tensor([[ 0.6248,  0.5922,  1.6515],
        [ 0.2860,  0.3323, -1.2482]])
tensor([[-2.0114, -0.8176,  0.4484],
        [-0.4285,  0.7101, -0.4121],
        [-2.0015,  0.1160, -0.8217]])
tensor([[-4.8161,  0.1013, -1.3210],
        [ 1.7808, -0.1427,  1.0170]])
torch.Size([2, 3])


In [47]:
print("Print 5 random numbers")
print(torch.rand(5))

print("Set the starting point to 15678 (this could be any integer number)")
torch.manual_seed(15678)

print("Print 5 random numbers")
print(torch.rand(5))

print("Print 5 more random numbers")
print(torch.rand(5))

print("Set the starting point to 15678 again")
torch.manual_seed(15678)

print("Print 5 random numbers")
print(torch.rand(5))

Print 5 random numbers
tensor([0.5446, 0.2227, 0.4673, 0.1460, 0.4769])
Set the starting point to 15678 (this could be any integer number)
Print 5 random numbers
tensor([0.6280, 0.7428, 0.1772, 0.4848, 0.6624])
Print 5 more random numbers
tensor([0.4395, 0.9383, 0.1213, 0.3481, 0.1033])
Set the starting point to 15678 again
Print 5 random numbers
tensor([0.6280, 0.7428, 0.1772, 0.4848, 0.6624])


In [0]:
# Broadcasting
# https://stackoverflow.com/questions/51371070/how-does-pytorch-broadcasting-work
# https://mc.ai/broadcasting-with-pytorch/